In [8]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import MinMaxScaler

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'College.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
college = pd.read_csv(file_content_stream)
college.head()

## disabling the 'FutureWarning' warning message
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [9]:
## Changing the variable 'Private' from a categorical to a numerical variable
college['Private'] = np.where(college['Private'] == 'No', 0 ,1)
college.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Abilene Christian University,1,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,1,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,1,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,1,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,1,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [10]:
## defining the input (x) and target (y) variables
X = college[['Private', 'F.Undergrad', 'P.Undergrad', 'Outstate', 'Room.Board', 'Books', 'Personal', 'S.F.Ratio', 'Grad.Rate']]
Y = college['Apps']
## splitting the data into 80% trianing and 20% testing datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2)

In [11]:
## transforming the input variables in the training and testing dataset to a 0-1 scale
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## building linear regression model


In [12]:
## fitting the data to a linear regression model
lm_md = LinearRegression().fit(X_train, Y_train)

In [13]:
## predicting on the testing dataset
lm_preds = lm_md.predict(X_test)
lm_preds

array([-3.90520372e+02,  1.65455560e+04,  2.66821328e+03,  6.09452302e+03,
        3.04051181e+02,  1.40162972e+03,  1.75449401e+03,  6.82608575e+03,
        1.11422302e+04,  6.92764957e+03, -1.76754715e+03,  3.61277455e+03,
        9.26457549e+03,  2.18812165e+03, -4.98890442e+02,  5.06773631e+03,
        1.65561824e+02,  9.00392483e+03,  7.76812946e+03,  2.29981560e+02,
        2.24166591e+03, -4.02227818e+02,  3.96666634e+03,  4.49990473e+03,
        7.13085612e+02,  3.21197623e+03, -8.04248355e+02,  2.51947189e+03,
       -1.25014857e+02,  4.83458301e+03, -8.66283970e+02, -2.69921836e+02,
        7.82366140e+02,  1.05784432e+03,  3.28763326e+03,  2.44650480e+03,
       -3.14787071e+01,  1.74173725e+03,  6.45421889e+02,  1.50399743e+03,
        1.21510110e+03,  3.88021052e+02, -5.97611242e+02,  1.03503202e+03,
        3.01955224e+03,  3.04009745e+03,  3.24555697e+03,  4.82267749e+03,
       -2.46697578e+03, -1.66755756e+02,  5.64988193e+03,  1.69337734e+04,
        1.02122301e+04,  

In [14]:
## calculating the MSE for this model
mse1 = np.mean((Y_test - lm_preds)**2)
print('The MSE of the Linear Regression Model is', mse1)

The MSE of the Linear Regression Model is 5347360.573749425


## building ridge regression model

In [15]:
## estimating the best lambda for ridge model (1st) and LASSO model (2nd)
ridge_cv = RidgeCV(alphas = np.linspace(0.001, 100, num = 100), cv = 5).fit(X_train, Y_train)
lasso_cv = LassoCV(alphas = np.linspace(0.001, 100, num = 100), normalize = True, cv = 5).fit(X_train, Y_train)

## extracting the best lambda value for both models
CV_lambda = ridge_cv.alpha_
cv_lambda = lasso_cv.alpha_
print('The optimal lambda for the ridge model is', CV_lambda)
print('The optimal lambda for the lasso model is', cv_lambda)

The optimal lambda for the ridge model is 1.011090909090909
The optimal lambda for the lasso model is 1.011090909090909


In [19]:
## building the ridge model
ridge_md = Ridge(alpha = CV_lambda).fit(X_train, Y_train)

## building the lasso model and capturing coefficients
lasso_md = Lasso(alpha = cv_lambda, normalize = True).fit(X_train, Y_train)
lasso_md.coef_
              
## predicting on the testing dataset for the ridge model
ridge_pred = ridge_md.predict(X_test)
lasso_pred = lasso_md.predict(X_test)

## computing the MSE of both models
mse2 = np.mean(np.power(Y_test - ridge_pred, 2))
mse3 = np.mean(np.power(Y_test - lasso_pred, 2))

## printing MSE results
print('The MSE of the Linear Regression Model is', mse1)
print('The MSE of the Ridge Regression Model is', mse2)
print('The MSE of the LASSO Regression Model is', mse3)

The MSE of the Linear Regression Model is 5347360.573749425
The MSE of the Ridge Regression Model is 4110162.056210365
The MSE of the LASSO Regression Model is 5000514.7275081


In [24]:
mse1>mse3>mse2

True

Based on these MSE values, I would choose the Ridge Regression model to predict the number of applications a university will receive because it has the lowest MSE value.